# Imports

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from applefy.detections.contrast import Contrast
from applefy.utils.positions import center_subpixel
from applefy.utils.file_handling import load_adi_data
from applefy.utils import flux_ratio2mag, mag2flux_ratio

from s4hci.utils.setups import contrast_grid_setup_1

# Load the data

In [18]:
datasets = [
    "HD22049_303_199_C-0065_C_",
    "HD22049_310_0100_C-0656_A_"
]

dataset_name = datasets[1]

In [19]:
dataset_file = Path("/fast/mbonse/s4/30_data/" + dataset_name + ".hdf5")
experiment_root_dir = Path("/fast/mbonse/s4/70_results/10_contrast_grids/" + dataset_name)

In [20]:
# we need the psf template for contrast calculation
science_data, angles, raw_psf_template_data = load_adi_data(
    dataset_file,
    data_tag="object_stacked_05",
    psf_template_tag="psf_template",
    para_tag="header_object_stacked_05/PARANG")

psf_template = np.median(raw_psf_template_data, axis=0)

dit_psf_template = 0.0042560
dit_science = 0.08
fwhm = 3.6
pixel_scale=0.02718

In [21]:
# we want toe image to show the innermost 1.2 arcsec
print(1.2 / pixel_scale * 2)

88.30022075055187


In [22]:
# we cut the image to 91 x 91 pixel to be slightly larger than 1.2 arcsec
cut_off = int((science_data.shape[1] - 91) / 2)
science_data = science_data[:, cut_off:-cut_off, cut_off:-cut_off]

# Design the full contrast grid

In [23]:
contrast_instance = Contrast(
    science_sequence=science_data,
    psf_template=psf_template,
    parang_rad=angles,
    psf_fwhm_radius=fwhm / 2,
    dit_psf_template=dit_psf_template,
    dit_science=dit_science,
    scaling_factor=1., # A factor to account e.g. for ND filters
    checkpoint_dir=experiment_root_dir)

In [24]:
experiment_root_dir

PosixPath('/fast/mbonse/s4/70_results/10_contrast_grids/HD22049_310_0100_C-0656_A_')

## Design fake planet experiments

In [25]:
# get fake planet setup
flux_ratios, separations, num_fake_planets = contrast_grid_setup_1(fwhm)
print("Brightness of fake planets in mag: " + str(flux_ratio2mag(flux_ratios)))
print("Separations of interest in lambda /D: " + str(separations / fwhm))

Brightness of fake planets in mag: [ 5.   5.5  6.   6.5  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5
 12.  12.5 13.  13.5 14.  14.5 15. ]
Separations of interest in lambda /D: [ 1.5  2.   2.5  3.   3.5  4.   4.5  5.   6.   7.   8.   9.  10.  11.
 12. ]


In [26]:
print("# Grid experiments: " + str(len(separations) * len(flux_ratios) * num_fake_planets))

# Grid experiments: 945


In [27]:
contrast_instance.design_fake_planet_experiments(
    flux_ratios=flux_ratios,
    num_planets=num_fake_planets,
    separations=separations,
    overwrite=True)

# Write out the job lists - 01 small selection with stacking

In [28]:
selection_file = experiment_root_dir.parent / Path("01_pca.txt")
selection_file

PosixPath('/fast/mbonse/s4/70_results/10_contrast_grids/01_pca.txt')

In [29]:
with open(selection_file, "w") as text_file:
    for dataset_name in datasets:
        num_exp = 0

        for tmp_key, values in contrast_instance.experimental_setups.items():

            text_file.write(dataset_name + " " + tmp_key + " 1\n")
            num_exp += 1
            if num_exp >5:
                break

# Write out the job lists - 02 all with stacking

In [30]:
for dataset_name in datasets:
    selection_file = experiment_root_dir.parent / Path("02_pca_" + dataset_name + ".txt")

    with open(selection_file, "w") as text_file:   

            for tmp_key, values in contrast_instance.experimental_setups.items():
                text_file.write(dataset_name + " " + tmp_key + " 1\n")

# Write out the job lists - 03 small selection without stacking

In [31]:
selection_file = experiment_root_dir.parent / Path("03_pca.txt")
selection_file

PosixPath('/fast/mbonse/s4/70_results/10_contrast_grids/03_pca.txt')

In [32]:
with open(selection_file, "w") as text_file:
    for dataset_name in datasets:
        num_exp = 0

        for tmp_key, values in contrast_instance.experimental_setups.items():
            text_file.write(dataset_name + " " + tmp_key + " 0\n")
            num_exp += 1
            if num_exp >5:
                break

# Write out the job lists - 04 all without stacking

In [33]:
for dataset_name in datasets:
    selection_file = experiment_root_dir.parent / Path("04_pca_" + dataset_name + ".txt")

    with open(selection_file, "w") as text_file:   

            for tmp_key, values in contrast_instance.experimental_setups.items():
                text_file.write(dataset_name + " " + tmp_key + " 0\n")